In [25]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [44]:
df = pd.read_excel("dropoutData/stud_2016_2018.xlsx");
df.head()
# id studente lu = 5938
# p = dat.loc[(dat['DataNascita'] == "29/01/1999") & (dat['Genere'] == 2) & (dat['voto_scuola_superiore'] == 93)] id bianca

In [27]:
df.describe()

,ID_Studente,Coorte,Genere,Diploma_scuola_superiore,voto_scuola_superiore,area_geografica_scuolasuperiore,area_geografica_residenza,Classe_ISEE,Merito_ISEE,OFA_assegnati,OFA_superati,CdS,Ambito,Campus,Abbandoni,Passaggi,Trasferimenti
count,67280.000000,67280.000000,67280.000000,67280.000000,55791.000000,67280.000000,67280.000000,67280.000000,24281.000000,67280.000000,67280.000000,67280.000000,67280.000000,67280.000000,67280.000000,67280.000000,67280.000000
mean,33640.500000,2017.003478,1.551813,16.960092,81.958129,15.748172,34.443861,1.583234,23718.498563,0.152467,0.117568,6791.541573,8.535791,1.614492,0.094441,0.020021,0.008413
std,19422.207393,0.818755,0.497312,33.301928,11.853937,33.754617,45.672541,1.744700,17033.256699,0.359476,0.322099,3375.063257,5.366329,1.197969,0.292444,0.140072,0.091334
min,1.000000,2016.000000,1.000000,1.000000,60.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,659.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,16820.750000,2016.000000,1.000000,2.000000,73.000000,1.000000,1.000000,0.000000,11767.640000,0.000000,0.000000,8004.000000,4.000000,1.000000,0.000000,0.000000,0.000000
50%,33640.500000,2017.000000,2.000000,3.000000,82.000000,2.000000,3.000000,1.000000,20716.620000,0.000000,0.000000,8531.000000,9.000000,1.000000,0.000000,0.000000,0.000000
75%,50460.250000,2018.000000,2.000000,6.000000,92.000000,4.000000,99.000000,3.000000,31990.270000,0.000000,0.000000,8873.000000,14.000000,1.000000,0.000000,0.000000,0.000000
max,67280.000000,2018.000000,2.000000,99.000000,100.000000,99.000000,99.000000,8.000000,303195.610000,1.000000,1.000000,9257.000000,16.000000,5.000000,1.000000,1.000000,1.000000


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67280 entries, 0 to 67279
Data columns (total 20 columns):
ID_Studente                        67280 non-null int64
Coorte                             67280 non-null int64
DataNascita                        67280 non-null datetime64[ns]
Genere                             67280 non-null int64
Diploma_scuola_superiore           67280 non-null int64
voto_scuola_superiore              55791 non-null float64
area_geografica_scuolasuperiore    67280 non-null int64
area_geografica_residenza          67280 non-null int64
Classe_ISEE                        67280 non-null int64
Merito_ISEE                        24281 non-null float64
OFA_assegnati                      67280 non-null int64
OFA_superati                       67280 non-null int64
CdS                                67280 non-null int64
TipoCorso                          67280 non-null object
Ambito                             67280 non-null int64
Sede                               67

In [29]:
boolean = pd.isnull(df)
pd.options.mode.use_inf_as_na = True
for col in df.columns:
    na = int(df[col].isna().sum())
    print("Column '" + col + "' has " + str(na) + " missing values")

Column 'ID_Studente' has 0 missing values
Column 'Coorte' has 0 missing values
Column 'DataNascita' has 0 missing values
Column 'Genere' has 0 missing values
Column 'Diploma_scuola_superiore' has 0 missing values
Column 'voto_scuola_superiore' has 11489 missing values
Column 'area_geografica_scuolasuperiore' has 0 missing values
Column 'area_geografica_residenza' has 0 missing values
Column 'Classe_ISEE' has 0 missing values
Column 'Merito_ISEE' has 42999 missing values
Column 'OFA_assegnati' has 0 missing values
Column 'OFA_superati' has 0 missing values
Column 'CdS' has 0 missing values
Column 'TipoCorso' has 0 missing values
Column 'Ambito' has 0 missing values
Column 'Sede' has 0 missing values
Column 'Campus' has 0 missing values
Column 'Abbandoni' has 0 missing values
Column 'Passaggi' has 0 missing values
Column 'Trasferimenti' has 0 missing values


In [30]:
# FILLING SU VALORI MANCANTI DI voto_scuola_superiore
# per ogni studente senza voto, calcolo la mediana del voto tra l'insieme di studenti che hanno in comune area_geografica_scuola_superiore e Diploma_scuola_superiore
df = pd.read_excel("dropoutData/stud_2016_2018.xlsx");

voto_scuola_superiore_mean = round(df['voto_scuola_superiore'].mean(), 2)

def calculate_mean_vote_with_similar_area_n_diploma(dt, area_scuola, area_residenza, diploma):
    if(diploma == 9):
        return voto_scuola_superiore_mean
    similar_student = dt.loc[(dt['Diploma_scuola_superiore'] == diploma) & ((dt['area_geografica_residenza'] == area_residenza) | (dt['area_geografica_scuolasuperiore'] == area_scuola))]#(dt['area_geografica_residenza'] == area_residenza | dt['area_geografica_scuolasuperiore'] == area_scuola)]
    return round(similar_student['voto_scuola_superiore'].mean(), 2)
    

cacheMap = {"11": 81}

students_without_vote = df.loc[df['voto_scuola_superiore'].isna()]

print(students_without_vote.shape)


for index, row in students_without_vote.iterrows():
    key = str(row.area_geografica_scuolasuperiore) + str(row.Diploma_scuola_superiore)
    if key in cacheMap:
        df.loc[index, 'voto_scuola_superiore'] =  cacheMap[key]
    else:
        median_vote = calculate_mean_vote_with_similar_area_n_diploma(df, row.area_geografica_scuolasuperiore, row.area_geografica_residenza,row.Diploma_scuola_superiore)
        df.loc[index, 'voto_scuola_superiore'] = median_vote
        cacheMap[key] = median_vote

#pd.options.mode.use_inf_as_na = True
print(df['voto_scuola_superiore'].isna().sum())

(11489, 20)
0


In [46]:
'''
Trasformazioni da effettuare nel dataset:
    - modifica data di nascita in anno di nascita (int64)
    - one hot-econding su (coorte, Diploma_scuola_superiore, area_geografica_scuolasuperiore, area_geografica_residenza
    - filling su voto_scuola_superiore (utilizzando la mediana dei valori dell'intorno calcolato su area_geografica_scuola_superiore e Diploma_scuola_superiore)
    - filling Merito_ISEE (la classe_ISEE in questo caso agisce come indicatore di reddito famigliare) a questo punto, si
      si potrebbe procedere creando una feature continua, la contribuzione_TASSE, che indichi la contribuzione delle tasse,
      quindi tutti gli studenti con classe_ISEE (0, 1) avranno contribuzione_TASSE pari a 0. L'implementazione sarà congrua
      con gli scaglioni delle tasse di unibo nel 2016/2017 - 2017/2018 - 2018/2019
    - modifica il tipo di corso con durata_corso -> L: 3, LM: 2, LMCU: 5
      
      COORTE 2016:
          Classe_ISEE -> contribuzione_TASSE
          [0, 1] -> 1
          Per le classi successive non c'è corrispondenza diretta quindi si effettua una codifica hard-coded su merito_ISEE
          [0, 23.000] -> 0
          [23.000,01, 28.880,63] -> 0,5
          [28.880,64, 33.371,84] -> 0,6
          [33.371,85, 38.506,71] -> 0,7
          [38.506,72, 44.922,89] -> 0,8
          [44.922,89, 51.337,89] -> 0,9
          [51.337,89, 100.000,00] -> 1
      
      COORTE 2017 e 2018:
          Classe_ISEE -> contribuzione_TASSE
          [0, 1] -> max || 1
          [0.00, 23.000]   -> 0
          [23.001, 33.000] -> (Merito_ISEE - 23.000) * 9% || .5
          [33.001, 45.000] -> (Merito_ISEE - 13.000) * 5% || .7
          [45.001, 60.000] -> (Merito_ISEE + 15.000) * 2,67% || .8
          [60.001, 70.000] -> (Merito_ISEE - 13.000) * 6% || .9
          [> 70.001] -> (Merito_ISEE - 13.000) * 7% || 1
          
          -> Già normalizzato tra 0 e 1
      
      A questo punto Classe_ISEE è un buon indicatore del reddito familiare e contribuzione_tasse del contributo pagato
    
Effettua un check sulla varianza di ogni feature e correlazione tra queste 
    (es: area_geografica_scuolasuperiore, area_geografica_residenza dovrebbe trasportare le medesime informazioni)
'''
# DATA PREPARATION
dataset = df

# ESTRAPOLAZIONE ANNO DI NASCITA
def get_year(date):
    return date.year

dataset["AnnoNascita"] = df["DataNascita"].apply(get_year)
dataset.drop("DataNascita", axis=1, inplace=True)

def one_hot_encode_n_merge(dt, column):
    encoded = pd.get_dummies(dt[column], prefix=column)
    return pd.concat([dt, encoded], axis=1)
    
# ONE HOT ENCODING Coorte
dataset = one_hot_encode_n_merge(dataset, "Coorte")
dataset.drop("Coorte", axis=1, inplace=True)

# ONE HOT ENCODING Diploma_scuola_superiore
dataset = one_hot_encode_n_merge(dataset, "Diploma_scuola_superiore")
dataset.drop("Diploma_scuola_superiore", axis=1, inplace=True)

# ONE HOT ENCODING area_geografica_scuolasuperiore
dataset = one_hot_encode_n_merge(dataset, "area_geografica_scuolasuperiore")
dataset.drop("area_geografica_scuolasuperiore", axis=1, inplace=True)

# ONE HOT ENCODING area_geografica_residenza
dataset = one_hot_encode_n_merge(dataset, "area_geografica_residenza")
dataset.drop("area_geografica_residenza", axis=1, inplace=True)

# FILLING Merito_ISEE

def calculate_contribuzione_tasse(dt):
    for index, row in dt.iterrows():
        if(row['Coorte'] == 2016):
            if(row['Classe_ISEE'] == 0 or row.Classe_ISEE == 1) : dt[index, "contribuzione_tasse"] = 1
            elif(row.Merito_ISEE <= 23000) : dt[index, "contribuzione_tasse"] = 0
            elif(row.Merito_ISEE <= 28880.63) : dt[index, "contribuzione_tasse"] = 0.5
            elif(row.Merito_ISEE <= 33371.84) : dt[index, "contribuzione_tasse"] = 0.6
            elif(row.Merito_ISEE <= 38506.71) : dt[index, "contribuzione_tasse"] = 0.7
            elif(row.Merito_ISEE <= 44922.89) : dt[index, "contribuzione_tasse"] = 0.8
            elif(row.Merito_ISEE <= 51337.89) : dt[index, "contribuzione_tasse"] = 0.9
            elif(row.Merito_ISEE >= 51337.90) : dt[index, "contribuzione_tasse"] = 1
        else:
            if(row.Classe_ISEE == 0 or row.Classe_ISEE == 1) : dt[index, "contribuzione_tasse"] = 1
            elif(row.Merito_ISEE <= 23000) : dt[index, "contribuzione_tasse"] = 0
            elif(row.Merito_ISEE <= 33000) : dt[index, "contribuzione_tasse"] = 0.5
            elif(row.Merito_ISEE <= 45000) : dt[index, "contribuzione_tasse"] = 0.7
            elif(row.Merito_ISEE <= 60000) : dt[index, "contribuzione_tasse"] = 0.8
            elif(row.Merito_ISEE <= 70000) : dt[index, "contribuzione_tasse"] = 0.9
            elif(row.Merito_ISEE >= 70001) : dt[index, "contribuzione_tasse"] = 1
                
dataset = calculate_contribuzione_tasse(dataset)
dataset.info()
    
    

KeyError: 'DataNascita'

In [35]:
'''
Guarda anche nel dataset riguardante gli esami:
    - calcola per ogni studente il numero di cfu
    - calcola per ogni studente la media dei voti degli esami
    - calcola giorni dall'ultimo esame

'''
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67280 entries, 0 to 67279
Data columns (total 41 columns):
ID_Studente                           67280 non-null int64
Genere                                67280 non-null int64
voto_scuola_superiore                 67280 non-null float64
Classe_ISEE                           67280 non-null int64
Merito_ISEE                           24281 non-null float64
OFA_assegnati                         67280 non-null int64
OFA_superati                          67280 non-null int64
CdS                                   67280 non-null int64
TipoCorso                             67280 non-null object
Ambito                                67280 non-null int64
Sede                                  67280 non-null object
Campus                                67280 non-null int64
Abbandoni                             67280 non-null int64
Passaggi                              67280 non-null int64
Trasferimenti                         67280 non-null int64


In [33]:
total = len(df)
dropout = (df["Abbandoni"].values == 1).sum()
ratio = round(dropout / total, 2)

print("Total row: " + str(total))
print("Total dropout: " + str(dropout))
print("Ratio: " + str(ratio * 100) + "%")
print("Base precision on dummy classifier: " + str(1 - ratio))

Total row: 67280
Total dropout: 6354
Ratio: 9.0%
Base precision on dummy classifier: 0.91
